# OpenAI Agents SDK: 시작하기

이 튜토리얼은 OpenAI Agents SDK를 활용하여 AI 에이전트를 구축하는 방법을 단계별로 안내한다. 기본적인 에이전트 생성부터 구조화된 출력, 대화형 에이전트까지 실습을 통해 학습할 수 있다.

---

## 1. Hello World - 첫 번째 에이전트 만들기

이 챕터에서는 가장 간단한 형태의 에이전트를 생성하고 실행하는 방법을 다룬다. OpenAI Agents SDK의 핵심 패턴인 **Agent -> Runner -> Result** 구조를 이해하는 것이 목표이다.

### 학습 내용

- 최소 구성으로 에이전트를 생성하는 방법
- `Runner.run_sync()`를 사용한 동기 방식 에이전트 실행
- 에이전트 응답에서 최종 출력값에 접근하는 방법
- 환경 변수 설정 및 API 키 검증 방법

In [1]:
# 필요한 패키지 설치
%pip install -q openai-agents python-dotenv pydantic


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /opt/miniconda3/envs/graph/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### 환경 설정

먼저 필요한 모듈을 임포트하고 환경 변수를 로드한다. `dotenv` 라이브러리를 사용하면 `.env` 파일에 저장된 API 키를 안전하게 불러올 수 있다. API 키가 올바르게 설정되었는지 검증하는 것은 프로덕션 환경에서 매우 중요한 보안 관행이다.

In [1]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner

# .env 파일에서 환경 변수를 로드한다
#!echo "OPENAI_API_KEY=sk-proj-발급받은 OpenAI API Key를 붙혀넣는다" >> .env
load_dotenv()

MODEL = "gpt-4o-mini"

### 에이전트 생성

에이전트는 `Agent` 클래스를 통해 생성한다. 가장 기본적인 에이전트는 `name`과 `instructions` 두 가지 파라미터만으로 구성할 수 있다. `name`은 에이전트의 식별자 역할을 하며, `instructions`는 에이전트가 어떻게 동작해야 하는지를 정의하는 시스템 프롬프트이다.

In [2]:
# Step 1: 에이전트 블루프린트 생성
agent = Agent(
    name="Assistant", 
    instructions="당신은 도움이 되는 어시스턴트입니다.",    
)

### 에이전트 실행

`Runner.run_sync()` 메서드를 사용하여 에이전트를 동기 방식으로 실행한다. 첫 번째 인자로 실행할 에이전트를, 두 번째 인자로 사용자 프롬프트를 전달한다. 실행 결과는 `RunResult` 객체로 반환되며, `final_output` 속성을 통해 에이전트의 최종 응답에 접근할 수 있다.

In [19]:
from agents import Runner

# Step 2: 프롬프트와 함께 에이전트 실행 (Jupyter에서는 await)
result = await Runner.run(agent, "프로그래밍에서의 재귀에 관한 하이쿠를 써줘.")

# Step 3: 결과 출력
print(result.final_output)

문을 두드려  
스스로를 다시 봐  
끝에 멈춘다


#### 참고
만약에 jupyter notebook이 아니고 python repper 실행이라면 아래의 `run_sync()` 메서드를 사용하자.
```python
# Step 2: 프롬프트와 함께 에이전트 실행
result = Runner.run_sync(agent, "프로그래밍에서의 재귀에 관한 하이쿠를 써줘.")

# Step 3: 결과 출력
print(result.final_output)
```

---

## 2. 하이쿠 작성 에이전트 - 고급 설정

이 챕터에서는 에이전트의 고급 설정 옵션을 활용하여 특정 도메인에 특화된 에이전트를 구축하는 방법을 다룬다. 모델 파라미터를 조정하여 에이전트의 동작을 세밀하게 제어할 수 있다.

### 학습 내용

- `ModelSettings`를 사용한 모델 파라미터 커스터마이징 (temperature, max_tokens)
- 특화된 작업을 위한 상세한 에이전트 지시문 작성
- `RunResult` 객체를 통한 실행 결과 모니터링 및 디버깅
- 비용 관리를 위한 토큰 사용량 추적
- 프로덕션 환경을 위한 오류 처리 패턴

### ModelSettings 이해하기

`ModelSettings` 클래스는 LLM의 동작을 제어하는 다양한 파라미터를 설정할 수 있게 해준다. `temperature`는 응답의 창의성 수준을 조절하며, 값이 높을수록 더 다양하고 창의적인 응답을 생성한다. `max_tokens`는 응답의 최대 길이를 제한하여 불필요한 토큰 소비를 방지한다.

In [20]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, RunResult, ModelSettings

### 특화된 에이전트 생성

하이쿠 작성에 특화된 에이전트를 생성한다. `model` 파라미터로 사용할 OpenAI 모델을 지정하고, `model_settings`로 모델의 동작 방식을 세부 조정한다. 시(詩) 창작과 같은 창의적 작업에는 높은 temperature 값(0.8)을 사용하여 다양한 표현을 유도한다. `instructions`에는 에이전트가 따라야 할 구체적인 규칙과 지침을 상세하게 기술한다.

In [21]:
# 상세 설정을 갖춘 하이쿠 작성 전문 에이전트 생성
haiku_agent = Agent(
    name="HaikuWriter",
    model=MODEL,                    # 사용할 OpenAI 모델 지정
    model_settings=ModelSettings(      # 모델 동작 설정
        temperature=0.8,               # 시 창작을 위한 높은 창의성
        max_tokens=100                 # 응답 길이 제한
    ),
    instructions="""
    당신은 자연과 인간 감정에 대한 깊은 감수성을 지닌 하이쿠 명인입니다.
    
    하이쿠를 작성할 때:
    1. 전통적인 5-7-5 음절 패턴을 엄격히 따르세요
    2. 가능하면 계절적 참조나 자연 이미지를 포함하세요
    3. 깨달음의 순간이나 감정적 통찰을 만들어내세요
    4. 단순하고 연상적인 언어를 사용하세요
    5. 적절한 줄바꿈과 함께 하이쿠를 제시하세요
    
    아름다움과 정확함으로 시간 속 한 순간을 포착하는 데 집중하세요.
    """
)

### 에이전트 실행 및 결과 분석

에이전트를 실행하고 반환된 `RunResult` 객체를 통해 다양한 실행 정보에 접근한다. `final_output`은 에이전트의 최종 응답 텍스트를 담고 있으며, `last_agent`를 통해 마지막으로 실행된 에이전트 정보에 접근할 수 있다. `raw_responses`에는 API 응답의 원시 데이터가 포함되어 있어 토큰 사용량 등 세부 정보를 확인할 수 있다.

In [24]:
# 창작 과제 정의
user_prompt = "겨울 숲의 고요함에 대한 하이쿠를 써주세요."

print(f"프롬프트: {user_prompt}")
print(f"에이전트: {haiku_agent.name} (모델: {haiku_agent.model})")
print("하이쿠 생성 중...")

# 에이전트 실행 및 포괄적인 결과 캡처
result = await Runner.run(haiku_agent, user_prompt)

# 포맷팅된 결과 출력
print("\n" + "="*50)
print("생성된 하이쿠")
print("="*50)
print(result.final_output)
print("="*50)

프롬프트: 겨울 숲의 고요함에 대한 하이쿠를 써주세요.
에이전트: HaikuWriter (모델: gpt-4o-mini)
하이쿠 생성 중...

생성된 하이쿠
눈 덮인 숲길,  
하얀 숨소리 들려,  
고요한 겨울.  


### 실행 세부 정보 확인

프로덕션 환경에서는 에이전트의 실행 정보를 모니터링하는 것이 중요하다. 토큰 사용량은 API 비용과 직결되므로 이를 추적하면 비용 관리에 도움이 된다. `raw_responses` 리스트에서 usage 정보를 추출하여 입력 토큰, 출력 토큰, 총 토큰 수를 확인할 수 있다.

In [25]:
# 학습을 위한 기술적 세부사항 검사
print("\n실행 세부 정보:")
print(f"   - 사용된 모델: {result.last_agent.model}")
print(f"   - 총 턴 수: {len(result.raw_responses)}")
print(f"   - 응답 항목 수: {len(result.new_items)}")

# 비용 모니터링을 위한 API 사용량 정보 접근
if result.raw_responses:
    usage = result.raw_responses[0].usage
    print(f"   - 사용된 토큰: {usage.total_tokens} (입력: {usage.input_tokens}, 출력: {usage.output_tokens})")

print("\n하이쿠 생성 완료!")


실행 세부 정보:
   - 사용된 모델: gpt-4o-mini
   - 총 턴 수: 1
   - 응답 항목 수: 1
   - 사용된 토큰: 210 (입력: 183, 출력: 27)

하이쿠 생성 완료!


---

## 3. 구조화된 출력 에이전트

이 챕터에서는 Pydantic 모델을 사용하여 에이전트로부터 구조화된 데이터를 얻는 방법을 다룬다. 일반 텍스트 대신 타입이 지정된 객체를 반환받아 데이터베이스, API 및 구조화된 워크플로우와 쉽게 통합할 수 있다.

### 학습 내용

- Pydantic `BaseModel` 클래스를 사용한 응답 스키마 정의
- `output_type` 파라미터를 통한 구조화된 응답 강제
- 원시 텍스트 대신 타입이 지정된 데이터에 접근하는 방법
- Pydantic `Field`를 사용한 필드 검증 및 문서화
- 저장 및 API 통합을 위한 JSON 변환

### Pydantic 모델 정의

구조화된 출력을 위해 먼저 응답 스키마를 Pydantic 모델로 정의해야 한다. `BaseModel`을 상속받아 클래스를 만들고, 각 필드에 타입 힌트를 지정한다. `Field` 함수를 사용하면 각 필드에 대한 설명을 추가할 수 있어 LLM이 더 정확하게 데이터를 채울 수 있도록 가이드한다.

In [26]:
import os
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import List
from agents import Agent, Runner, ModelSettings

In [27]:
class TaskBreakdown(BaseModel):
    """작업 분해 응답을 위한 구조체이다."""
    main_task: str = Field(description="완수해야 할 주요 작업")
    subtasks: List[str] = Field(description="주요 작업을 완료하기 위해 필요한 하위 작업 목록")
    estimated_time: str = Field(description="모든 작업을 완료하는 데 예상되는 시간")
    difficulty_level: str = Field(description="쉬움, 보통, 어려움, 전문가 중 하나")
    required_skills: List[str] = Field(description="작업 완료에 필요한 기술 목록")

### 구조화된 출력 에이전트 생성

에이전트를 생성할 때 `output_type` 파라미터에 앞서 정의한 Pydantic 모델을 전달하면, 에이전트는 자동으로 해당 스키마에 맞는 구조화된 데이터를 반환한다. 구조화된 응답의 일관성을 높이기 위해 낮은 temperature 값(0.2)을 사용하는 것이 좋다.

In [28]:
print("=== 구조화된 출력 에이전트 데모 ===")
print("이 에이전트는 Pydantic 모델을 사용하여 구조화된 데이터를 반환한다.\n")

# 구조화된 데이터를 출력하는 에이전트 생성
planning_agent = Agent(
    name="TaskPlanner",
    model=MODEL,
    model_settings=ModelSettings(temperature=0.2),  # 더 일관된 구조를 위한 낮은 temperature
    instructions="""
    당신은 전문 프로젝트 플래너입니다. 작업이나 프로젝트가 주어지면,
    시간 추정과 함께 관리 가능한 하위 작업으로 분해하세요.
    
    복잡성을 분석하고 작업 완료에 필요한 기술과 시간에 대한
    현실적인 평가를 제공하세요.
    """,
    output_type=TaskBreakdown  # 에이전트에게 구조화된 데이터를 반환하도록 지시
)

=== 구조화된 출력 에이전트 데모 ===
이 에이전트는 Pydantic 모델을 사용하여 구조화된 데이터를 반환한다.



### 에이전트 실행 및 구조화된 결과 처리

에이전트를 실행하면 `result.final_output`이 문자열이 아닌 `TaskBreakdown` 객체로 반환된다. 이 객체의 각 속성에 직접 접근하여 데이터를 처리할 수 있다. Pydantic 모델은 `model_dump_json()` 메서드를 제공하여 JSON 형식으로 쉽게 직렬화할 수 있어 API 응답이나 데이터베이스 저장에 유용하다.

In [33]:
# 분해할 예시 작업
user_task = "내 프로그래밍 프로젝트를 보여줄 개인 웹사이트 포트폴리오 구축"

print(f"분석할 작업: {user_task}\n")
print("계획 에이전트 실행 중...")

# 에이전트 실행
result = await Runner.run(planning_agent, user_task)

# result.final_output은 이제 문자열이 아닌 TaskBreakdown 객체이다
task_plan = result.final_output

분석할 작업: 내 프로그래밍 프로젝트를 보여줄 개인 웹사이트 포트폴리오 구축

계획 에이전트 실행 중...


In [34]:
# 구조화된 결과 출력
print("\n=== 작업 분해 결과 ===")
print(f"주요 작업: {task_plan.main_task}")
print(f"난이도: {task_plan.difficulty_level}")
print(f"예상 시간: {task_plan.estimated_time}")

print("\n하위 작업:")
for i, subtask in enumerate(task_plan.subtasks, 1):
    print(f"  {i}. {subtask}")

print("\n필요 기술:")
for skill in task_plan.required_skills:
    print(f"  - {skill}")


=== 작업 분해 결과 ===
주요 작업: 개인 웹사이트 포트폴리오 구축
난이도: 중간
예상 시간: 4-6주

하위 작업:
  1. 목표 및 요구사항 정의
  2. 도메인 이름 선택 및 등록
  3. 웹 호스팅 서비스 선택
  4. 웹사이트 디자인 및 레이아웃 구상
  5. 프로그래밍 언어 및 프레임워크 선택
  6. 프론트엔드 개발 (HTML, CSS, JavaScript)
  7. 백엔드 개발 (필요시)
  8. 포트폴리오 콘텐츠 작성 (프로젝트 설명, 기술 스택 등)
  9. SEO 최적화 및 웹사이트 성능 테스트
  10. 웹사이트 배포 및 유지보수 계획 수립

필요 기술:
  - HTML/CSS
  - JavaScript
  - 웹 디자인
  - 서버 관리
  - SEO 최적화
  - Git 및 버전 관리


### 기술적 세부 정보 확인

반환된 객체가 실제로 Pydantic 모델의 인스턴스인지 확인하고, JSON으로 직렬화하여 출력한다. 이 기능은 구조화된 데이터를 외부 시스템과 통합할 때 매우 유용하다.

In [35]:
print("\n=== 기술적 세부 정보 ===")
print(f"응답 타입: {type(task_plan)}")
print(f"Pydantic 모델 여부: {isinstance(task_plan, BaseModel)}")
print(f"원시 JSON:\n{task_plan.model_dump_json(indent=2)}")


=== 기술적 세부 정보 ===
응답 타입: <class '__main__.TaskBreakdown'>
Pydantic 모델 여부: True
원시 JSON:
{
  "main_task": "개인 웹사이트 포트폴리오 구축",
  "subtasks": [
    "목표 및 요구사항 정의",
    "도메인 이름 선택 및 등록",
    "웹 호스팅 서비스 선택",
    "웹사이트 디자인 및 레이아웃 구상",
    "프로그래밍 언어 및 프레임워크 선택",
    "프론트엔드 개발 (HTML, CSS, JavaScript)",
    "백엔드 개발 (필요시)",
    "포트폴리오 콘텐츠 작성 (프로젝트 설명, 기술 스택 등)",
    "SEO 최적화 및 웹사이트 성능 테스트",
    "웹사이트 배포 및 유지보수 계획 수립"
  ],
  "estimated_time": "4-6주",
  "difficulty_level": "중간",
  "required_skills": [
    "HTML/CSS",
    "JavaScript",
    "웹 디자인",
    "서버 관리",
    "SEO 최적화",
    "Git 및 버전 관리"
  ]
}


---

## 4. 대화형 에이전트

이 챕터에서는 대화형 인터페이스를 구축하는 방법을 다룬다. `run_demo_loop` 함수를 사용하면 사용자와 지속적인 대화 세션을 유지하는 챗봇을 쉽게 프로토타이핑할 수 있다.

### 학습 내용

- `run_demo_loop`를 사용한 연속 대화 세션 구현
- `async/await`를 활용한 비동기 에이전트 실행
- 자연스러운 대화를 유지하는 대화형 에이전트 구축
- 실시간 대화형 시나리오에서의 사용자 입력 처리

### 주의사항

`run_demo_loop`은 터미널 기반의 대화형 세션을 위해 설계되었으므로, Jupyter 노트북 환경에서는 정상적으로 동작하지 않을 수 있다. 이 코드는 Python 스크립트(`.py` 파일)로 실행하는 것을 권장한다.

In [36]:
import os
import asyncio
from dotenv import load_dotenv
from agents import Agent, run_demo_loop

### 대화형 에이전트 정의

대화형 에이전트는 사용자와의 자연스러운 대화 흐름을 유지하도록 설계된다. `instructions`에 대화적이고 친근한 톤을 유지하며, 적절한 후속 질문을 통해 대화를 이어나가도록 지시한다.

In [37]:
# 대화형 에이전트 생성
conversational_agent = Agent(
    name="FriendlyAssistant",
    instructions="""
    당신은 친근하고 도움이 되는 AI 어시스턴트입니다. 응답을 대화체로
    유지하고 흥미롭게 만드세요. 대화가 자연스럽게 흘러가도록
    적절한 경우 후속 질문을 하세요.
    """
)

print(f"대화형 에이전트 '{conversational_agent.name}'가 생성되었다.")

대화형 에이전트 'FriendlyAssistant'가 생성되었다.


### 대화 루프 실행 (터미널 전용)

`run_demo_loop` 함수는 에이전트와의 연속적인 대화 세션을 시작한다. 사용자가 'quit'을 입력할 때까지 대화가 계속된다. 이 기능은 터미널 환경에서 실행해야 하므로, 아래 코드는 `.py` 파일로 저장하여 실행하는 것을 권장한다.

```python
# 이 코드를 interactive_demo.py로 저장하여 실행하라
# python interactive_demo.py

import os
import asyncio
from dotenv import load_dotenv
from agents import Agent, run_demo_loop

load_dotenv()

async def main():
    conversational_agent = Agent(
        name="FriendlyAssistant",
        instructions="""
        당신은 친근하고 도움이 되는 AI 어시스턴트입니다. 응답을 대화체로
        유지하고 흥미롭게 만드세요. 대화가 자연스럽게 흘러가도록
        적절한 경우 후속 질문을 하세요.
        """
    )
    
    print("대화형 대화를 시작한다. 종료하려면 'quit'을 입력하라.")
    await run_demo_loop(conversational_agent)

if __name__ == "__main__":
    asyncio.run(main())
```

### Jupyter에서 단일 대화 테스트

Jupyter 환경에서는 `run_demo_loop` 대신 `Runner.run_sync`를 사용하여 단일 대화 턴을 테스트할 수 있다.

In [38]:
from agents import Runner

# 단일 대화 턴 테스트
test_message = "안녕하세요! 오늘 뭘 도와드릴까요?"

print(f"사용자: {test_message}")
result = await Runner.run(conversational_agent, test_message)
print(f"\n에이전트: {result.final_output}")

사용자: 안녕하세요! 오늘 뭘 도와드릴까요?

에이전트: 안녕하세요! 반갑습니다. 저는 오늘 어떤 일을 도와드릴 수 있을까요? 궁금한 점이 있으시거나, 오늘 하고 싶은 주제가 있으실까요? 아니면 그냥 수다 떨고 싶으실 수도 있고요 😊


---

## 마무리

이 튜토리얼에서는 OpenAI Agents SDK의 핵심 개념과 사용법을 다루었다. 다음과 같은 내용을 학습하였다:

1. **기본 에이전트 생성**: `Agent` 클래스와 `await Runner.run()` 또는 `Runner.run_sync()`를 사용한 간단한 에이전트 구축
2. **고급 설정**: `ModelSettings`를 통한 모델 파라미터 커스터마이징과 토큰 사용량 모니터링
3. **구조화된 출력**: Pydantic 모델을 활용한 타입 안전한 응답 처리
4. **대화형 에이전트**: `run_demo_loop`를 사용한 연속 대화 세션 구현

이러한 기본 패턴을 바탕으로 더 복잡한 에이전트 시스템을 구축할 수 있다. 도구(Tools) 통합, 멀티 에이전트 협업, 핸드오프(Handoff) 등 고급 주제는 후속 튜토리얼에서 다룰 예정이다.